In [1]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import RussianStemmer
import pandas as pd
import re
import tensorflow as tf
import numpy as np
import sys
import vk

In [2]:
t = TweetTokenizer()
stemer = RussianStemmer()
regex = re.compile('[^а-яА-Я ]')

In [3]:
tweets_col_number = 3
negative_tweets = pd.read_csv(
    'negative.csv', header=None, delimiter=';')[[tweets_col_number]]
positive_tweets = pd.read_csv(
    'positive.csv', header=None, delimiter=';')[[tweets_col_number]]

In [4]:
sess = tf.Session()
w = tf.get_variable("Weights_2", [1,310000], dtype=tf.float64, initializer=tf.glorot_uniform_initializer(
    seed=None,
    dtype=tf.float64
))
sess.run (tf.global_variables_initializer())
dictionary = []
expected = tf.placeholder(tf.float64)
x = tf.placeholder(tf.float64)
predict = tf.sigmoid(tf.reduce_sum(tf.multiply(w, x)))
loss = tf.pow(predict - expected,2)
Optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

In [ ]:
for i,tweet in positive_tweets.iterrows():
    for token in t.tokenize(tweet[3]):
        vector = np.zeros((1,310000))
        stem = stemer.stem(token)
        if stem not in dictionary: dictionary.append(stem)
        vector[0,dictionary.index(stem)] = 1.0
        sess.run([loss,Optimizer],feed_dict={x: vector,expected: 1.0})

    


print ('Positive is done !')


for i,tweet in negative_tweets.iterrows():
    for token in t.tokenize(tweet[3]):
        vector = np.zeros((1,310000))
        stem = stemer.stem(token)
        if stem not in dictionary: dictionary.append(stem)
        vector[0,dictionary.index(stem)] = 1.0
        sess.run([loss,Optimizer],feed_dict={x: vector,expected: 0.0})
        
   
print ('Negative is done !')
    


Positive is done !


In [ ]:
vector = np.zeros((1,310000))   
for i in t.tokenize('Текст'):
    
    
    print (stemer.stem(i))
    print (dictionary.index(stemer.stem(i)))
    vector[0,dictionary.index(stemer.stem(i))] = 1.0
print (sess.run(predict,feed_dict={x: vector}))
